In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Eric Meinhardt / emeinhardt@ucsd.edu**

# Introduction

This notebook will take 
 - a `.tsv` file specifying a binary or ternary feature matrix.
   - the file is assumed to have a header row indicating feature labels
   - each non-header row represents an object's feature vector
   - features are assumed to be {`+`,`-`,`0`} (following phonological convention) by default
 - a (potentially -- comma-separated -- list of) column name(s) indicating any columns not containing feature-value information (e.g. an object's label = IPA symbol in the case of phonological feature matrices).
 
and write
 - the ordered list of feature names (reflecting the ordering in the input `.tsv`) to a `.txt` file
 - a serialized numpy ndarray `.npy` file representing the unique feature vectors of the `.tsv` file as a matrix, with each row corresponding to an object and feature values represented as `1`, `-1`, or `0`
 - a `.tsv` with `+` replaced with `1` and `-` replaced with `-1`.

Note that if your feature matrix contains multiple objects that have the same featural description, this notebook will detect and record as much, but the final matrix it produces will only have one row for each unique feature vector.

# Parameters

In [2]:
from os import getcwd, chdir, listdir, path, mkdir, makedirs

**Note:** the cell below has a cell tag `parameters`, meaning this notebook will work with the `papermill` package.

In [3]:
# parameters cell

# i = ''
i = 'bakovic_chart_riggle_hayes.tsv'
# i = 'hayes.tsv'

# c = ''
c = 'symbol'

# f = ''
# # f = '+,-,0'
# # f = '1,-1,0'

# u = ''
# # u = 'tolerate'
# # u = 'raise_exception'

# o = 
o = 'brh.npy'
# o = 'hayes.npy'

# d = ''
d = 'brh_features.txt'
# d = 'hayes_features.txt'

In [4]:
output_dir = path.dirname(o)
if not path.exists(output_dir) and output_dir != '':
    print('Making output path {0}'.format(output_dir))
    makedirs(output_dir)

In [5]:
columns_to_remove = c.split(',')
columns_to_remove

['symbol']

# Imports / load data

In [6]:
from funcy import *

In [7]:
import os
from copy import deepcopy

In [8]:
import numpy as np

In [9]:
import csv

In [10]:
objects = []

with open(i, encoding='utf-8-sig') as csvfile:
    my_reader = csv.DictReader(csvfile, delimiter='\t', quoting=csv.QUOTE_NONE, quotechar='@')
    for row in my_reader:
        #print(row)
        objects.append(row)
len(objects)
objects[:5]

94

[OrderedDict([('symbol', 'p'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '-'),
              ('cont', '-'),
              ('del. rel.', '-'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '0'),
              ('low', '0'),
              ('back', '0'),
              ('front', '0'),
              ('approx', '-'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '0'),
              ('ATR', '0')]),
 OrderedDict([('symbol', 'b'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '+'),
              ('cont', '-'),
              ('de

# Check that every object is defined on the same set of features

In [11]:
features = lmap(lambda d: tuple(d.keys()),
                objects)
values = lmap(lambda d: set(d.values()),
              objects)

In [12]:
assert all(features[0] == features[i] for i in range(len(features)))

In [13]:
features = features[0]
features
original_fieldnames = deepcopy(features)

('symbol',
 'syll',
 'cons',
 'son',
 'labial',
 'coronal',
 'dorsal',
 'voice',
 'cont',
 'del. rel.',
 'ant',
 'dist',
 'strid',
 'high',
 'low',
 'back',
 'front',
 'approx',
 'nas',
 'lat',
 's.g.',
 'c.g.',
 'round',
 'ATR')

In [14]:
values

[{'+', '-', '0', 'p'},
 {'+', '-', '0', 'b'},
 {'+', '-', '0', 'ɸ'},
 {'+', '-', '0', 'β'},
 {'+', '-', '0', 'f'},
 {'+', '-', '0', 'v'},
 {'+', '-', '0', 't̪'},
 {'+', '-', '0', 'd̪'},
 {'+', '-', '0', 'θ'},
 {'+', '-', '0', 'ð'},
 {'+', '-', '0', 't'},
 {'+', '-', '0', 'd'},
 {'+', '-', '0', 'ʦ'},
 {'+', '-', '0', 'ʣ'},
 {'+', '-', '0', 's'},
 {'+', '-', '0', 'z'},
 {'+', '-', '0', 'ʈ'},
 {'+', '-', '0', 'ɖ'},
 {'+', '-', '0', 'ʂ'},
 {'+', '-', '0', 'ʐ'},
 {'+', '-', '0', 'ʃ'},
 {'+', '-', '0', 'ʒ'},
 {'+', '-', '0', 'ʧ'},
 {'+', '-', '0', 'ʤ'},
 {'+', '-', '0', 'c'},
 {'+', '-', '0', 'ɟ'},
 {'+', '-', '0', 'ç'},
 {'+', '-', '0', 'ʝ'},
 {'+', '-', '0', 'k'},
 {'+', '-', '0', 'ɡ'},
 {'+', '-', '0', 'x'},
 {'+', '-', '0', 'ɣ'},
 {'+', '-', '0', 'q'},
 {'+', '-', '0', 'ɢ'},
 {'+', '-', '0', 'χ'},
 {'+', '-', '0', 'ʁ'},
 {'+', '-', '0', 'ħ'},
 {'+', '-', '0', 'ʕ'},
 {'+', '-', '0', 'm'},
 {'+', '-', '0', 'ʙ'},
 {'+', '-', '0', 'ɱ'},
 {'+', '-', '0', 'ⱱ'},
 {'+', '-', '0', 'n̪'},
 {'+', '

# Check uniqueness of objects

In [15]:
def areIdentical(a, b):
    for k in features:
        if k != 'symbol':
            if a[k] != b[k]:
                return False
    return True

In [16]:
def get_matches(obj):
    return [o for o in objects if areIdentical(obj, o)]

In [17]:
def hasDuplicates(obj):
    return len(get_matches(obj)) > 1

In [18]:
objectsWithDuplicates = [o for o in objects if hasDuplicates(o)]
len(objectsWithDuplicates)

6

In [19]:
for obj in objectsWithDuplicates:
    print(obj['symbol'])
    for m in get_matches(obj):
        print('\t{0}'.format(m['symbol']))
    print('\n')

m
	m
	ɱ


ɱ
	m
	ɱ


w
	w
	ɥ


ɥ
	w
	ɥ


ə
	ə
	ɜ


ɜ
	ə
	ɜ




# Remap +/-/0 to +1/-1/0

In [20]:
valueRemap = {'+':1,
              '-':-1,
              '0':0}

def remapValue(v):
    return valueRemap[v]

remapValue('-')

def remapValues(d):
    for k in d:
        if d[k] in valueRemap:
            d[k] = remapValue(d[k])
    return d

-1

In [21]:
objects = [remapValues(o) for o in objects]
objects

[OrderedDict([('symbol', 'p'),
              ('syll', -1),
              ('cons', 1),
              ('son', -1),
              ('labial', 1),
              ('coronal', -1),
              ('dorsal', -1),
              ('voice', -1),
              ('cont', -1),
              ('del. rel.', -1),
              ('ant', 0),
              ('dist', 0),
              ('strid', 0),
              ('high', 0),
              ('low', 0),
              ('back', 0),
              ('front', 0),
              ('approx', -1),
              ('nas', -1),
              ('lat', -1),
              ('s.g.', -1),
              ('c.g.', -1),
              ('round', 0),
              ('ATR', 0)]),
 OrderedDict([('symbol', 'b'),
              ('syll', -1),
              ('cons', 1),
              ('son', -1),
              ('labial', 1),
              ('coronal', -1),
              ('dorsal', -1),
              ('voice', 1),
              ('cont', -1),
              ('del. rel.', -1),
              ('ant', 0),
    

In [22]:
objects_with_remapped_values = deepcopy(objects)

# Remove undesired columns

In [23]:
features = tuple([f for f in features if f not in columns_to_remove])
features

('syll',
 'cons',
 'son',
 'labial',
 'coronal',
 'dorsal',
 'voice',
 'cont',
 'del. rel.',
 'ant',
 'dist',
 'strid',
 'high',
 'low',
 'back',
 'front',
 'approx',
 'nas',
 'lat',
 's.g.',
 'c.g.',
 'round',
 'ATR')

In [24]:
for obj in objects:
    for c in columns_to_remove:
        del obj[c]

# Identify feature/dimension labels

In [25]:
dimension_labels = tuple(k for k in features)
dimension_labels

('syll',
 'cons',
 'son',
 'labial',
 'coronal',
 'dorsal',
 'voice',
 'cont',
 'del. rel.',
 'ant',
 'dist',
 'strid',
 'high',
 'low',
 'back',
 'front',
 'approx',
 'nas',
 'lat',
 's.g.',
 'c.g.',
 'round',
 'ATR')

# Project off feature labels + convert each object to a tuple of values + unique-ify the set of objects

In [26]:
def toTuple(o):
    return tuple([o[f] for f in dimension_labels])

In [27]:
from random import choice

In [28]:
random_object = choice(objects)
random_object

OrderedDict([('syll', -1),
             ('cons', 1),
             ('son', 1),
             ('labial', -1),
             ('coronal', 1),
             ('dorsal', -1),
             ('voice', 1),
             ('cont', -1),
             ('del. rel.', 0),
             ('ant', 1),
             ('dist', -1),
             ('strid', 0),
             ('high', 0),
             ('low', 0),
             ('back', 0),
             ('front', 0),
             ('approx', 1),
             ('nas', -1),
             ('lat', 1),
             ('s.g.', -1),
             ('c.g.', -1),
             ('round', 0),
             ('ATR', 0)])

In [29]:
toTuple(random_object)

(-1, 1, 1, -1, 1, -1, 1, -1, 0, 1, -1, 0, 0, 0, 0, 0, 1, -1, 1, -1, -1, 0, 0)

In [30]:
objects_tupled = tuple([toTuple(o) for o in objects])
objects_tupled

((-1,
  1,
  -1,
  1,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0),
 (-1,
  1,
  -1,
  1,
  -1,
  -1,
  1,
  -1,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0),
 (-1,
  1,
  -1,
  1,
  -1,
  -1,
  -1,
  1,
  1,
  0,
  1,
  -1,
  0,
  0,
  0,
  0,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0),
 (-1,
  1,
  -1,
  1,
  -1,
  -1,
  1,
  1,
  1,
  0,
  1,
  -1,
  0,
  0,
  0,
  0,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0),
 (-1,
  1,
  -1,
  1,
  -1,
  -1,
  -1,
  1,
  1,
  0,
  -1,
  1,
  0,
  0,
  0,
  0,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0),
 (-1,
  1,
  -1,
  1,
  -1,
  -1,
  1,
  1,
  1,
  0,
  -1,
  1,
  0,
  0,
  0,
  0,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0),
 (-1,
  1,
  -1,
  -1,
  1,
  -1,
  -1,
  -1,
  -1,
  1,
  1,
  -1,
  0,
  0,
  0,
  0,
  -1,
  -1,
  -1,
  -1,
  -1,
  0,
  0),
 (-1,
  1,
  -1,
  -1,
  1,
  -1,
  1,
  -1,
  -1,
  1,
  1,
  -1,
  0,
  0,
  0,
  0,
  -1,
  -1,
  -1,
  -1,

In [31]:
len(objects_tupled)
len(set(objects_tupled))

94

91

In [32]:
unique_objects = set(objects_tupled)

In [33]:
unique_objects_np = np.array([o for o in unique_objects], dtype='int8')
unique_objects_np.shape
unique_objects_np.dtype
unique_objects_np

(91, 23)

dtype('int8')

array([[-1, -1, -1, ..., -1,  0,  0],
       [-1,  1,  1, ..., -1,  0,  0],
       [-1,  1, -1, ..., -1,  0,  0],
       ...,
       [-1,  1, -1, ..., -1,  0,  0],
       [ 1, -1,  1, ..., -1, -1,  1],
       [-1,  1,  1, ..., -1, -1,  0]], dtype=int8)

# Export

In [34]:
with open(os.path.splitext(i)[0] + '_remapped.tsv', 'w', newline='\n') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=original_fieldnames, delimiter='\t', quoting=csv.QUOTE_NONE, quotechar='@')

    writer.writeheader()
    writer.writerows(objects_with_remapped_values)

In [40]:
objects_with_remapped_values

[OrderedDict([('symbol', 'p'),
              ('syll', -1),
              ('cons', 1),
              ('son', -1),
              ('labial', 1),
              ('coronal', -1),
              ('dorsal', -1),
              ('voice', -1),
              ('cont', -1),
              ('del. rel.', -1),
              ('ant', 0),
              ('dist', 0),
              ('strid', 0),
              ('high', 0),
              ('low', 0),
              ('back', 0),
              ('front', 0),
              ('approx', -1),
              ('nas', -1),
              ('lat', -1),
              ('s.g.', -1),
              ('c.g.', -1),
              ('round', 0),
              ('ATR', 0)]),
 OrderedDict([('symbol', 'b'),
              ('syll', -1),
              ('cons', 1),
              ('son', -1),
              ('labial', 1),
              ('coronal', -1),
              ('dorsal', -1),
              ('voice', 1),
              ('cont', -1),
              ('del. rel.', -1),
              ('ant', 0),
    

In [35]:
%%capture

with open(d, 'w') as the_file:
    for seq in dimension_labels:
        the_file.write(seq + '\n')

In [36]:
d

'brh_features.txt'

In [41]:
!cat 'brh_features.txt'

syll
cons
son
labial
coronal
dorsal
voice
cont
del. rel.
ant
dist
strid
high
low
back
front
approx
nas
lat
s.g.
c.g.
round
ATR


In [42]:
o

'brh.npy'

In [43]:
np.save(o, unique_objects_np)